In [ ]:
def sift_knn(img1,img2):
    global infoAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    # ratio test as per Lowe's paper
    j=0
    k=0    
    for i, (m1,m2) in enumerate(matches):
        k+=1
        if m1.distance < 0.7 * m2.distance:
            j+=1
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
            info = [lambda pt1, pt2: [10*pt2[0], 10*pt2[1], 10*(pt2[1]-pt1[1]), 10*(pt2[0]-pt1[0]), img2[pt2[0]-5 : pt2[0]+5,pt2[1]-5 : pt2[1]+5].flatten()]]
            infolist.append(info)
#             print(np.array(info).shape)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)
        
#     print(j)
#     print(k)
    # Draw match in blue, error in red
    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 0)
    res = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
#     cv2.imshow("res",res)
    try:
        infoAll.append(infolist)
    except:
        time.sleep(0.001*random.randint(1,9))
        infoAll.append(infolist)
        

In [ ]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans



kernel = np.ones((5,5),np.uint8)
infoAll = []
scr = None
# Gkp1=[None,None,None,None,None,None]
# Gkp2 = [None,None,None,None,None,None]
# Gdes1 = [None,None,None,None,None,None]
# Gdes2 = [None,None,None,None,None,None]
Prevlist = [[],[],[],[],[],[]]
def sift_knn(img1,img2,part):
    global infoAll, scr
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)
#     if not(Gkp1[part] == None):
#         kp1 += Gkp1[part]
#         kp2+= Gkp2[part]
#         des1 +=Gdes1[part]
#         des2 +=Gdes2[part]
    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    # ratio test as per Lowe's paper
    j=0
    k=0    
    for i, (m1,m2) in enumerate(matches):
        k+=1
        if m1.distance < 0.5 * m2.distance:
            j+=1
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 10*(pt2[1]-pt1[1]), 10*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
#             print(np.array(info).shape)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)
        
#     print(j)
#     print(k)
    # Draw match in blue, error in red
    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 0)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)
#     cv2.imshow("res",res)
#     infolist = np.array(infolist).reshape((-1,4))
    
    
    
    try:
        infoAll+=infolist
#         print(np.array(infoAll).shape)
    except:
        time.sleep(0.001*random.randint(1,9))
#         print("1.",np.array(infoAll).shape)
        infoAll+=infolist
#         print("2.",np.array(infoAll).shape)
    nc = min(np.array(infolist).shape[0]//2,5)
    
    if nc>=2:
        kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=200)
        
        temp = infolist
        infolist+=Prevlist[part]
        Prevlist[part] = temp
        
        kmeans.fit(infolist)
        centroid = kmeans.cluster_centers_
        for cnt in centroid:
#             print("centroid",cnt)
            cv2.circle(img2,(int(cnt[0]/10),int(cnt[1]/10)),5,(0,0,255),-1)
#     Gkp1[part] = kp1
#     Gkp2[part] = kp2
#     Gdes1[part] = des1
#     Gdes2[part] = des2    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global infoAll, scr
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*6*8
    prev1 = []
    shape = [260,680]

    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
    while True:
 
        start_time = time.time()
        infoAll = []
        ret, scr = vrec.read()
        ret, scr = vrec.read()
#         scr1 = copy.deepcopy(scr)
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
    
#         frame = cv2.GaussianBlur(scr, (15, 15), 0)
#         scr = cv2.addWeighted(scr,30,frame,-29,0)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        scr = gray
        cv2.imshow('scred',scr)
        scr1 = scr[0:shape[0]//2, 0:shape[1]//3]
        scr2 = scr[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = scr[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = scr[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = scr[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = scr[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == []:
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,1))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,2))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,3))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,4))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,5))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
# #         sift_knn(prev4,scr4)
#         canny = cv2.Canny(gray,50,100)
#         canny = blank-canny
#         im2, contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         blank = cv2.drawContours(blank, contours, -1, 0, 1)
#         dist_transform = cv2.distanceTransform(canny,cv2.DIST_L2,5)
#         ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
# #         scr = cv2.bilateralFilter(scr,20,40,40)
# #     blank = cv2.dilate(blank,kernel,iterations = 2)
#         blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,5)
#         cv2.imshow("blk1",blank)
#         blank = np.uint8(blank)
#         im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         scr1 = cv2.drawContours(scr1, contours1, -1, 0, 1)
#         cv2.imshow("blk2",scr1)
        
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()

        
#         print("final",np.array(infoAll).shape)

#         scr = cv2.drawContours(scr, contours1, -1,[0,255,255], 3)
        cv2.imshow('print',scr)

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
running
FPS:  3.420088341801393
FPS:  9.156849346470153


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:152: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  6.226226933530864
FPS:  5.844367714229182
FPS:  7.261267219156999
FPS:  6.020650165864498
FPS:  7.551222708917322
FPS:  5.198774893497292
FPS:  7.6219607044987026
FPS:  4.966353440436496
FPS:  6.5174991647825715
FPS:  5.455444578703457
FPS:  6.939375038259756
FPS:  4.960685220382042
FPS:  6.794584813567449
FPS:  5.58434154414359
FPS:  6.334887992073657
FPS:  7.276383653090504
FPS:  6.072013434477967
FPS:  5.516171838524919
FPS:  5.729806083208677
FPS:  5.758282250701885
FPS:  5.67779175087753
FPS:  5.192512627513123
FPS:  5.974681380544235


Exception in thread Thread-84005:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-92-f1dd402a2840>", line 45, in sift_knn
    matches = flann.knnMatch(des1,des2,k=2)
cv2.error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/flann/src/miniflann.cpp:487: error: (-215) (size_t)knn <= index_->size() in function runKnnSearch_




FPS:  4.649257048478903
FPS:  7.784167401289844


...

FPS:  4.660135838558107
FPS:  4.682323889731115
FPS:  5.099798770738468
FPS:  4.348560486124287
FPS:  4.8864387270549745
FPS:  5.69625180116986
FPS:  3.8349570221531804
FPS:  5.1651572955945415
FPS:  3.6071129106190556
FPS:  4.775800719847969
FPS:  3.2447178522360165
FPS:  4.192291699984008
FPS:  3.311367573055601
FPS:  4.38006702249086
FPS:  3.397848347375243
FPS:  3.986442894590274
FPS:  2.934187735228688
FPS:  4.450123817789444
FPS:  2.844099193145363
FPS:  4.02171993253512
FPS:  3.1101678953683063
FPS:  3.9436087892663387
FPS:  3.112792322632308
FPS:  3.5513859477254233
FPS:  4.194610206545078
FPS:  3.8688058856193055
FPS:  4.957735824078146
FPS:  3.0186967142205288


Exception in thread Thread-84557:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-92-f1dd402a2840>", line 45, in sift_knn
    matches = flann.knnMatch(des1,des2,k=2)
cv2.error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/flann/src/miniflann.cpp:315: error: (-210) type=0
 in function buildIndex_




FPS:  4.171390551699514
FPS:  3.238259650379777
FPS:  3.7900599913794584
FPS:  4.171415443461726
FPS:  3.9355864199268487
FPS:  3.5552511593567786
FPS:  3.759003837598427
FPS:  3.3474254465703748
FPS:  4.024726186843535
FPS:  3.360432770577754
FPS:  4.225913835489461
FPS:  2.6973814016363185
FPS:  4.286949260467973
FPS:  2.535529907895723
FPS:  4.154483277782455
FPS:  2.469592476119877
FPS:  3.560025021919643
FPS:  2.2145251550028853
FPS:  3.683430593793773
FPS:  2.5182121869273795
FPS:  4.19162136232811
FPS:  2.6658276973023955
FPS:  3.689817405104493
FPS:  2.888119972924977
FPS:  3.4477065864178096
FPS:  2.4480564051082108
FPS:  3.847353508001897
FPS:  2.5652607887268815


Exception in thread Thread-84725:
Traceback (most recent call last):
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/shreyashkawalkar/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-92-f1dd402a2840>", line 45, in sift_knn
    matches = flann.knnMatch(des1,des2,k=2)
cv2.error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/flann/src/miniflann.cpp:487: error: (-215) (size_t)knn <= index_->size() in function runKnnSearch_




FPS:  2.7181236585340565
FPS:  3.0314426134721018
FPS:  2.715304688966718
FPS:  2.972634467586883
FPS:  2.4712177613120425
FPS:  2.389370861700546
FPS:  2.8485591844773603
FPS:  2.6980008323700613
FPS:  2.3961686931315382
FPS:  2.623694737665181
FPS:  2.7447244447963626
FPS:  3.0343506424920097
FPS:  2.820064169627273
FPS:  3.172045669911841
FPS:  2.942074650610117
FPS:  2.8798765189663955
FPS:  2.526471514803243
FPS:  3.431167698641948
FPS:  2.9040151989320866
FPS:  2.861133224053333
FPS:  2.2849870777384083
FPS:  2.7148319146333333
FPS:  2.4423131233080753
FPS:  2.4229923808940623
FPS:  2.1774339912576703
FPS:  2.3749189590749697
FPS:  2.551304635568229
FPS:  2.4138726034957756
FPS:  2.3572245888925107
FPS:  2.2536935542803103
FPS:  2.1589508897434615
FPS:  2.335411888578748
FPS:  2.3007335593674045
FPS:  2.49024457725314
FPS:  2.211573272168835
FPS:  2.0888956399829675
FPS:  2.4152890420393374
FPS:  2.7796526149969747
FPS:  2.3322303213562656
FPS:  2.5892859237382777
FPS:  2.3715953